# Importing all the necessary libraries



In [1]:
pip install -U textblob

Note: you may need to restart the kernel to use updated packages.


In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')
stopword = stopwords.words('english')
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from nltk.corpus import stopwords
import string
import json
from nltk.sentiment.util import *
from textblob import TextBlob

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sharanyakarthikeyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sharanyakarthikeyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/sharanyakarthikeyan/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/sharanyakarthikeyan/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sharanyakarthikeyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Import and Read the data



In [53]:
with open('/Users/sharanyakarthikeyan/Downloads/tweets.json') as jfile:
  json_file = json.load(jfile)

# Convert into Data Frame 



In [54]:
df=pd.DataFrame(json_file).T


In [55]:
df.head(10)


,tweet_author,tweet_text
1374140386071961602,Hematopoiesis News,⚕️ Scientists conducted a Phase II study of ac...
1374032432173842437,"Michael Wang, MD",This phase 2 Acalabrutinib-Venetoclax (AV) tri...
1373902876553048065,1stOncology,#NICE backs #AstraZenecas #Calquence for #CLL ...
1373656782367813635,Toby Eyre,#acalabrutinib is a valuable option in pts int...
1372941634334232586,Lymphoma Hub,NICE has recommended the use of acalabrutinib ...
1372927482278539265,David Ledger,NICE backs AstraZeneca’s Calquence for CLL htt...
1372911723305394179,N Wales Cancer Forum,This is England for now - these decisions usua...
1372888121159868423,European Pharmaceutical Review,"AstraZeneca’s Calquence (acalabrutinib), a che..."
1372866915081797632,Graham Collins,Superstar @tobyeyre82 responding to the excell...
1372825553837944834,CLL Ireland,CLL patients all know the drug Ibrutinib and y...


# checking for null values and unique writters

print('total no of null valus in the data:\n',df.isnull().sum())
print('total no tweet author:',df.tweet_author.nunique())

In [66]:
def clean_data(df):
    df = df.lower()
    df = re.sub(r'[^(a-zA-Z)\s]','',df)
    #remove urls
    df = re.sub(r'http\S+', " ", df)
    #remove mentions
    df = re.sub(r'@\w+',' ',df)
    #remove hashtags
    df= re.sub(r'#\w+', ' ',df)
    #remove digits
    df = re.sub(r'\d+()', ' ', df)
    #remove html tags and umber
    df = re.sub('r<.*?>',' ', df)
    #remove stopwords
    df = df.split()
    df = " ".join([word for word in df if not word in stopword])
    return df
df['tweet_text']=df['tweet_text'].apply(lambda x:clean_data(x))
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df['tweet_text']= df['tweet_text'].apply(lambda x: remove_punct(x))
#Tokenization of the text data
def tokenization(text):
    text = re.split('\W+', text)
    return text
df['tweet_text'] = df['tweet_text'].apply(lambda x: tokenization(x.lower()))


# Stanford NLP NER



In [89]:
from nltk.tag.stanford import StanfordNERTagger
!wget 'https://nlp.stanford.edu/software/stanford-ner-2018-10-16.zip' -O stanford-ner-2018-10-16.zip
!unzip stanford-ner-2018-10-16.zip

nltk.download('punkt')



--2022-11-16 10:03:02--  https://nlp.stanford.edu/software/stanford-ner-2018-10-16.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-ner-2018-10-16.zip [following]
--2022-11-16 10:03:03--  https://downloads.cs.stanford.edu/nlp/software/stanford-ner-2018-10-16.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180358328 (172M) [application/zip]
Saving to: ‘stanford-ner-2018-10-16.zip’

stanford-ner-2018-1 100%[===================>] 172.00M  1.62MB/s    in 10m 53s 

2022-11-16 10:13:57 (270 KB/s) - ‘stanford-ner-2018-10-16.zip’ saved [180358328/180358328]

Archive:  stanford-ner-2018-10-16.z

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sharanyakarthikeyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [93]:
st = StanfordNERTagger('stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz',
                       'stanford-ner-2018-10-16/stanford-ner.jar',
                       encoding='utf-8')

# objective 2
Find out the sentiment/polarity of each author towards each of the entities



In [113]:
df['tweet_text'] = df['tweet_text'].astype('str')
def get_polarity(text):
  return TextBlob(text).sentiment.polarity
df['Polarity'] = df['tweet_text'].apply(get_polarity)

In [117]:
df.to_csv('objective2.csv')


In [115]:
df.head(10)


,tweet_author,tweet_text,Polarity,overall_Polarity
1374140386071961602,Hematopoiesis News,"['scientists', 'conducted', 'phase', 'ii', 'st...",0.000000,0.000000
1374032432173842437,"Michael Wang, MD","['phase', 'acalabrutinibvenetoclax', 'av', 'tr...",0.000000,0.000000
1373902876553048065,1stOncology,"['nice', 'backs', 'astrazenecas', 'calquence',...",0.600000,0.600000
1373656782367813635,Toby Eyre,"['acalabrutinib', 'valuable', 'option', 'pts',...",0.100000,0.100000
1372941634334232586,Lymphoma Hub,"['nice', 'recommended', 'use', 'acalabrutinib'...",0.600000,0.600000
1372927482278539265,David Ledger,"['nice', 'backs', 'astrazenecas', 'calquence',...",0.600000,0.600000
1372911723305394179,N Wales Cancer Forum,"['england', 'decisions', 'usually', 'come', 'w...",-0.250000,-0.250000
1372888121159868423,European Pharmaceutical Review,"['astrazenecas', 'calquence', 'acalabrutinib',...",0.000000,0.000000
1372866915081797632,Graham Collins,"['superstar', 'tobyeyre', 'responding', 'excel...",0.800000,0.800000
1372825553837944834,CLL Ireland,"['cll', 'patients', 'know', 'drug', 'ibrutinib...",0.045455,0.045455
